In [3]:
import numpy as np
import pandas as pd

In [4]:
#Tester
t = {'MID': ['A','A','B','B'], 'Day': [0, 5, 0, 5], 'Size': [10, 20, 11, 22]}
tf = pd.DataFrame(data=t)
tf.head()

tf_day0 = tf.loc[tf.Day == 0.0]
print(tf_day0)
tf_day0 = tf_day0[['MID', 'Size']]
print(tf_day0)
tf_day0 = tf_day0.rename(columns={"Size": "Day-0-Size"})
print(tf_day0)

d = tf
start_day = 0
end_day = 5
start_size = size_on_day(d, start_day)
end_size = size_on_day(d, end_day)
mids = pd.DataFrame(d.MID).drop_duplicates()
mids = mids.merge(start_size, on='MID', validate='one_to_one')
mids = mids.merge(end_size, on='MID', validate='one_to_one')
print(mids)

  MID  Day  Size
0   A    0    10
2   B    0    11
  MID  Size
0   A    10
2   B    11
  MID  Day-0-Size
0   A          10
2   B          11


NameError: name 'size_on_day' is not defined

In [8]:
df = pd.DataFrame({'Study': ['A', 'A', 'A', 'B', 'B', 'B'],
                   'Group': ['C', 'C', 'D', 'C', 'D', 'D']}, 
                   columns=['Study', 'Group'])
df

,Study,Group
0,A,C
1,A,C
2,A,D
3,B,C
4,B,D
5,B,D


In [9]:
gf = df.groupby(['Study', 'Group']).apply(lambda x: x.name).reset_index(name='Cohort')
df = df.merge(gf, on=['Study', 'Group'], validate='many_to_one')
cohort_names = df['Cohort'].unique()
cohort_names
cohort_dict = {}
for x in range(0, len(cohort_names)):
    cohort_dict[cohort_names[x]] = x
cohort_dict

,Study,Group,Cohort
0,A,C,"(A, C)"
1,A,C,"(A, C)"
2,A,D,"(A, D)"
3,B,C,"(B, C)"
4,B,D,"(B, D)"
5,B,D,"(B, D)"


In [14]:
cohort_names = df['Cohort'].unique()
cohort_names
cohort_dict = {}
for x in range(0, len(cohort_names)):
    cohort_dict[cohort_names[x]] = x
cohort_dict

{('A', 'C'): 0, ('A', 'D'): 1, ('B', 'C'): 2, ('B', 'D'): 3}

In [16]:
df['Cohort'] = df['Cohort'].map(cohort_dict)
df

,Study,Group,Cohort
0,A,C,0
1,A,C,0
2,A,D,1
3,B,C,2
4,B,D,3
5,B,D,3


In [13]:
gb = df.groupby(['Study', 'Group']).get_group(x).reset(name='group')

NameError: name 'x' is not defined

In [30]:
df = pd.DataFrame({'Study': ['A', 'A', 'A', 'B', 'B'],
                   'Size': [1, 2, 3, 4, 8]}, 
                   columns=['Study', 'Size'])
df

,Study,Size
0,A,1
1,A,2
2,A,3
3,B,4
4,B,8


In [33]:
sm = df.groupby('Study')['Size'].mean().reset_index(name='sample_mean')
sm

,Study,sample_mean
0,A,2
1,B,6


In [34]:
df = df.merge(sm, on=['Study'], validate='many_to_one')
df

,Study,Size,sample_mean
0,A,1,2
1,A,2,2
2,A,3,2
3,B,4,6
4,B,8,6


In [17]:
gf = df.groupby('A')['B'].mean().to_frame(name='mean').reset_index()

In [18]:
gf

,A,mean
0,1,2
1,2,4


In [19]:
df = df.merge(gf, on='A', validate='many_to_one')
df

,A,B,C,mean
0,1,0,1,2
1,1,2,2,2
2,1,4,1,2
3,2,3,1,4
4,2,5,2,4


In [24]:
d = pd.DataFrame({'MID': ['A', 'A', 'A', 'B', 'B'],
                   'Day': [1, 1, 3, -1, 2],
                   'Size': [1, 2, 3, 4, 8]}, 
                   columns=['MID', 'Day', 'Size'])
d

,MID,Day,Size
0,A,1,1
1,A,1,2
2,A,3,3
3,B,-1,4
4,B,2,8


In [37]:
start_d = d.loc[d.groupby('MID')['Day'].idxmin()].reset_index(drop=True)
start_d

,MID,Day,Size
0,A,1,1
1,B,-1,4


In [26]:
start_tf = pd.DataFrame({'MID': ['A', 'B'],
                   'Day': [1, -1],
                   'Size': [1, 4]}, 
                   columns=['MID', 'Day', 'Size'])
start_tf

,MID,Day,Size
0,A,1,1
1,B,-1,4


In [27]:
start_d.equals(start_tf)

True

In [33]:
end_d = d.loc[d.groupby('MID')['Day'].idxmax()].reset_index(drop=True)
end_d

,MID,Day,Size
0,A,3,3
1,B,2,8


In [34]:
end_tf = pd.DataFrame({'MID': ['A', 'B'],
                   'Day': [3, 2],
                   'Size': [3, 8]}, 
                   columns=['MID', 'Day', 'Size'])

In [35]:
end_d.equals(end_tf)

True

In [38]:
start_d = start_d.rename(columns = {'Size': 'start_size'})
end_d = end_d.rename(columns = {'Size': 'end_size'})
nmids = d.MID.nunique()
assert(nmids == start_d.MID.nunique())
assert(nmids == end_d.MID.nunique())
m = start_d.merge(end_d, on='MID', validate='one_to_one')
assert(nmids == m.MID.nunique())

In [39]:
m

,MID,Day_x,start_size,Day_y,end_size
0,A,1,1,3,3
1,B,-1,4,2,8


In [41]:
# could do this at the start to get a start_day and end_day for each mouse
# then eliminate mice w.o. a large enough gap between start_day and end_day
# can also get stats on distribution of start and end days, to help explain where the mice are going!